In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
from typing import NamedTuple
from dataclasses import dataclass
from enum import IntEnum
from typing import Callable
import hybrid.hybrid as hybrid
from statsmodels.stats import proportion

# Birth death extinction probability

A simple birth death process where we measure the extinction probability.

In [ ]:
seed = None
if seed is not None:
    rng = np.random.default_rng(seed)
else:
    rng = np.random.default_rng()

initial_pop = 10
y0 = np.array([initial_pop])


# each row of one of these matrices refers to a species
N = np.array([
    [1,-1]
])
rate_involvement_matrix = np.array([
    [1,1]
])

birth_rate = 11
death_rate = 10

from numba import jit, float64
from numba.types import Array

@jit(Array(float64, 1, "C")(float64), nopython=True)
def birth_death_rate_constants(t):
    return np.array([birth_rate, death_rate]).astype(float64)

options = {
    'jit':True,
    'approximate_rtot':True,
    'contrived_no_reaction_rate':100.0,
}

In [ ]:
extinctions = 0
trials = 1000
for i in range(trials):
    result = hybrid.forward_time(y0, [0, 15.0], birth_death_rate_constants, N, rate_involvement_matrix, rng, partition_function=hybrid.FixedThresholdPartitioner(1000).partition_function, **options, discontinuities=[4.0])
    extinctions += result.y[-1] == 0.0
    print("i", i, "Extinctions:", extinctions, "extinction probability:", extinctions/(i+1))
    #break
print("Observed exctinction probability:", extinctions/(i+1), "+/-", proportion.proportion_confint(extinctions, trials, alpha=0.05, method='jeffreys'))
print("Expected extinction probability:", (1/(birth_rate/death_rate))**initial_pop)
# expect 38.5%

The same as the last experiment with the addition of a high abundance species exponentially growing but decoupled from the birth death process. We check that the extinction probability is correct *and* that the exponential growth matches the analytic form.

In [ ]:
seed = None
if seed is not None:
    rng = np.random.default_rng(seed)
else:
    rng = np.random.default_rng()

initial_pop = 10
y0 = np.array([10e10,initial_pop])

# births and deaths: we have an irrelevant birth/death process for a large scale species (to test that our system is well isolated)
# and a birth death process for a small-valued species

# each row of one of these matrices refers to a species
N = np.array([
    [1,-1,0,0],
    [0,0,1,-1]
])
rate_involvement_matrix = np.array([
    [1,1,0,0],
    [0,0,1,1]
])

birth_rate = 11
death_rate = 10

from numba import jit, float64
from numba.types import Array

@jit(Array(float64, 1, "C")(float64), nopython=True)
def birth_death_rate_constants(t):
    return np.array([1.1, 1.0, birth_rate, death_rate])

In [ ]:
extinctions = 0
trials = 1000
for i in range(trials):
    result = hybrid.forward_time(y0, [0, 15.0], birth_death_rate_constants, N, rate_involvement_matrix, rng, partition_function=hybrid.FixedThresholdPartitioner(1000).partition_function, jit=True, discontinuities=[4.0])
    extinctions += result.y[-1] == 0.0
    print("i", i, "Extinctions:", extinctions, "extinction probability:", extinctions/(i+1))
    #break
print("Observed exctinction probability:", extinctions/(i+1), "+/-", proportion.proportion_confint(extinctions, trials, alpha=0.05, method='jeffreys'))
print("Expected extinction probability:", (1/(birth_rate/death_rate))**initial_pop)
# expect 38.5%

In [ ]:
print("Observed exctinction probability:", 417/(1000), "+/-", proportion.proportion_confint(417, trials, alpha=0.05, method='jeffreys'))


In [ ]:
import matplotlib.pyplot as plt

plt.scatter(result.t_history, result.y_history[0,:].T)
plt.plot(result.t_history, y0[0] * np.exp(result.t_history * 0.1))
plt.title("Deterministic exponential growth of plentiful species")
plt.legend(['Species count', 'analytic solution'])

In [ ]:
plt.plot(result.t_history, result.y_history[1,:].T)
plt.ylim(0, 200)
plt.title("Stochastic birth death process for scarce species")

# Race to rescue

In [ ]:
seed = None
if seed is not None:
    rng = np.random.default_rng(seed)
else:
    rng = np.random.default_rng()

y0 = np.array([
    1e9, # target cells
    1e2,  # wildtype infected cells
    0,     # mutant infected cells
])

# it's more natural to write down the reactions and then transpose (so that the rows of the final matrix refer to a species and the columns refer to reactions)
N = np.array([
    [1,0,0], # birth target cell
    [0,-1,0], # death of wildtype
    [0,0,-1], # death of mutant
    [-1,1,0], # infection --> new wildtypes (by wildtype)
    [-1,0,1], # infection --> mutation into mutant (by wildtype)
    [-1,0,1], # infection --> new mutants
]).T

rate_involvement_matrix = np.array([
    [0,0,0],
    [0,1,0],
    [0,0,1],
    [1,1,0],
    [1,1,0],
    [1,0,1],
]).T

b = 1001    # 0th order birth rate of target cells
R0 = 0.8   # wildtype is dying out
d  = 10    # death rate for wild type per day
beta = R0 * d / (y0[0]) # beta derived from above two
mu = 0.001 # rare mutations can save it
dmu = 2 # mutant dies less
Rmu = beta * y0[0] / dmu
assert np.isclose(Rmu, R0 * d/dmu) # Rmu should be x times bigger where x is d/dmu

def rescue_probability(R0, Rmu, mu, y0):
    return 1 - np.exp((mu*R0*(1-1/Rmu)*y0[1])/(R0-1))

@jit(nopython=True)
def k_of_t(t):
    return np.array([b, d, dmu, beta, beta*mu, beta])

1 - rescue_probability(R0, Rmu, mu, y0)

In [ ]:
extinctions = 0
for i in range(1000):
    result = hybrid.forward_time(y0, [0, 15.0], k_of_t, N, rate_involvement_matrix, rng, partition_function=hybrid.FixedThresholdPartitioner(1000).partition_function)
    extinctions += result.y[-1] == 0.0
    print("i", i, "Exctinctions:", extinctions, "extinction probability:", extinctions/(i+1))
print("Expected extinction probability:", 1-rescue_probability(R0, Rmu, mu, y0))

In [ ]:
print("Expected extinction probability:", 1-rescue_probability(R0, Rmu, mu, y0))

In [ ]:
fig, ax = plt.subplots(1,3)
fig.set_size_inches(14, 5)
ax[0].plot(result.t_history, result.y_history[1,:].T)
ax[0].set_title("Constant death rate for wildtype")

ax[1].plot(result.t_history, result.y_history[2,:].T)
ax[1].set_title("Mutant")

ax[2].plot(result.t_history, result.y_history[0,:].T)
ax[2].set_title("Target cells")

In [ ]:
result.y_history[:,-1]